<a href="https://colab.research.google.com/github/newerZGQ/colab/blob/main/index-tts-2-on-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 安装工具链
%pip install uv
!uv tool install "huggingface_hub[cli]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 81.0 MB/s eta 0:00:00
Resolved 18 packages in 221ms
Prepared 18 packages in 138ms
Installed 18 packages in 14ms
 + anyio==4.11.0
 + certifi==2025.11.12
 + click==8.3.1
 + filelock==3.20.0
 + fsspec==2025.10.0
 + h11==0.16.0
 + hf-xet==1.2.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + huggingface-hub==1.1.4
 + idna==3.11
 + packaging==25.0
 + pyyaml==6.0.3
 + shellingham==1.5.4
 + sniffio==1.3.1
 + tqdm==4.67.1
 + typer-slim==0.20.0
 + typing-extensions==4.15.0
Installed 2 executables: hf, tiny-agents


In [3]:
# ✅ Step 1: 获取最近n个提交
import requests
import ipywidgets as widgets
from IPython.display import display
import subprocess

# 仓库信息
repo = "index-tts/index-tts"
commits_url = f"https://api.github.com/repos/{repo}/commits"
response = requests.get(commits_url)
commits = response.json()

# 提取前两个提交 SHA 和信息
options = []
for commit in commits[:3]:
    sha = commit["sha"]
    message = commit["commit"]["message"]
    options.append((f"{message[:60]}... ({sha[:7]})", sha))  # 显示前60字+SHA前7位

# ✅ Step 2: 创建下拉菜单和按钮
dropdown = widgets.Dropdown(
    options=options,
    description='选择版本:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

button = widgets.Button(description="下载选定版本", button_style='success')
output = widgets.Output()

# ✅ Step 3: 下载并解压所选版本
def download_selected_version(sha):
    url = f"https://github.com/index-tts/index-tts/archive/{sha}.zip"
    subprocess.run(["wget", "-O", "main.zip", url])
    subprocess.run(["unzip", "main.zip"])
    subprocess.run(["mv", f"index-tts-{sha}", "index-tts"])
    subprocess.run(["rm", "main.zip"])

def on_button_clicked(b):
    with output:
        output.clear_output()
        print(f"正在下载版本：{dropdown.label}")
        download_selected_version(dropdown.value)
        print("✅ 下载完成，已解压到 index-tts 文件夹")

button.on_click(on_button_clicked)

# ✅ Step 4: 显示控件
display(dropdown, button, output)


Dropdown(description='选择版本:', layout=Layout(width='80%'), options=(('Merge pull request #537 from pandalee99/p…

Button(button_style='success', description='下载选定版本', style=ButtonStyle())

Output()

In [ ]:
# 安装依赖
!cd /content/index-tts && uv sync --all-extras

Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: .venv
Resolved 190 packages in 1ms
⠧ Preparing packages... (163/178)                                               

In [ ]:
# 下载模型 checkpoints
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

Fetching 20 files: 100% 20/20 [00:00<00:00, 1512.17it/s]
/content/index-tts/checkpoints


In [ ]:
# 为了直接利用 Gradio 的内网穿透功能
# 需要对 index—tts/webui.py 进行修改

# 如果因为 IndexTTS 官方 GitHub 仓库的更改造成本代码不可用
# 还可以手动修改 webui.py ，在 launch 函数调用中加上 `share = True`


%cd /content/index-tts

launch_line = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port)'
launch_line_with_share = 'demo.launch(server_name=cmd_args.host, server_port=cmd_args.port, share=True)'

with open("webui.py", encoding='utf8') as f:
  webui_with_share = []
  for line in f.readlines():
    if line.strip() == launch_line:
      webui_with_share.append(line.replace(launch_line, launch_line_with_share))
    else:
      webui_with_share.append(line)

  with open("webui_with_share.py", 'w', encoding='utf8') as f:
    f.writelines(webui_with_share)

/content/index-tts/index-tts


In [1]:
# 启动

%cd /content/index-tts
!uv run webui_with_share.py

[Errno 2] No such file or directory: '/content/index-tts/index-tts/'
/content
error: Failed to spawn: `webui_with_share.py`
  Caused by: No such file or directory (os error 2)
